<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [274]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_codigo.csv')



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [275]:
# parte a)
df1= pd.read_csv(archivos_anio[0])
df2= pd.read_csv(archivos_anio[1])

df1.columns = df1.columns.str.lower()
df2.columns = df2.columns.str.lower()

df_anio= pd.concat((df1,df2))
df_anio.head()

,codigo_iso,anio,indice,ranking
0,AFG,2001,35.5,59.0
1,AGO,2001,30.2,50.0
2,ALB,2001,NaN,NaN
3,AND,2001,NaN,NaN
4,ARE,2001,NaN,NaN


In [276]:
#Parte b)
df_dupliso = df_codigos[df_codigos.duplicated(subset = ['codigo_iso'],keep=False)]
df_duplpais = df_codigos[df_codigos.duplicated(subset = ['pais'],keep=False)]
df_dupliso.head()
df_duplpais.head()
df_codigos = df_codigos[(df_codigos['pais']!='malo') & (df_codigos['codigo_iso']!='NER')]


In [277]:
#Parte c)
df = pd.merge(df_anio, df_codigos , on='codigo_iso')
df.head()

,codigo_iso,anio,indice,ranking,pais
0,AFG,2001,35.5,59.0,Afghanistán
1,AGO,2001,30.2,50.0,Angola
2,ALB,2001,NaN,NaN,Albania
3,AND,2001,NaN,NaN,Andorra
4,ARE,2001,NaN,NaN,Emiratos Árabes Unidos




### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

In [278]:
# Parte 2 estructura del dataframe
print("la cantidad de filas y columnas son:")
df.shape

la cantidad de filas y columnas son:


(3043, 5)

In [279]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3043 entries, 0 to 3042
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   codigo_iso  3043 non-null   object 
 1   anio        3043 non-null   int64  
 2   indice      2648 non-null   float64
 3   ranking     2820 non-null   float64
 4   pais        3043 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 119.0+ KB


In [280]:
# Parte 2 Resumen estadístico
df.describe()

,anio,indice,ranking
count,3043.000000,2648.000000,2820.000000
mean,2009.941176,206.871412,480.533688
std,5.786029,2703.623037,6494.342299
min,2001.000000,0.000000,1.000000
25%,2005.000000,15.250000,34.750000
50%,2009.000000,28.045000,71.000000
75%,2015.000000,41.385000,110.000000
max,2019.000000,64536.000000,121056.000000


In [281]:
print(f"max de indice es:  {df['indice'].max()}")
print(f"min de indice es:  {df['indice'].min()}")
print(f"mean de indice es:  {df['indice'].mean()}")
paismininde = df[df['indice'] == df['indice'].min()]
paismaxinde = df[df['indice'] == df['indice'].max()]
print(f"El/los pais que es minimo en indice son/es:")
print(paismininde)
print(f"El/los pais que es maximo en indice son/es:")
print(paismaxinde)
print(f"max de ranking es:  {df['ranking'].max()}")
print(f"min de ranking es:  {df['ranking'].min()}")
print(f"mean de ranking es:  {df['indice'].mean()}")
paisminrank = df[df['ranking'] == df['ranking'].min()]
paismaxrank= df[df['ranking'] == df['ranking'].max()]
print(f"El/los pais que es minimo en ranking son/es:")
#print(paisminrank)
print(f"El/los pais que es maximo en ranking son/es:")
#print(paismaxrank)

max de indice es:  64536.0
min de indice es:  0.0
mean de indice es:  206.87141238670694
El/los pais que es minimo en indice son/es:
     codigo_iso  anio  indice  ranking          pais
1297        DNK  2008     0.0      2.0     Dinamarca
1306        FIN  2008     0.0      2.0     Finlandia
1328        IRL  2008     0.0      2.0       Irlanda
1374        NOR  2008     0.0      2.0       Noruega
1404        SWE  2008     0.0      2.0        Suecia
1460        CHE  2009     0.0      2.0         Suiza
1485        FIN  2009     0.0      2.0     Finlandia
1510        ISL  2009     0.0      2.0      Islandia
1552        NLD  2009     0.0      2.0  Países Bajos
1553        NOR  2009     0.0      2.0       Noruega
1583        SWE  2009     0.0      2.0        Suecia
El/los pais que es maximo en indice son/es:
     codigo_iso  anio   indice   ranking    pais
2058        KSV  2014  64536.0  120614.0  Kosovo
max de ranking es:  121056.0
min de ranking es:  1.0
mean de ranking es:  206.87141238670

In [282]:
#los valores nulos por columnas
df.isnull().sum()

,0
codigo_iso,0
anio,0
indice,395
ranking,223
pais,0


In [283]:
print(f"La cantidad de proporcion de datos nulos en codigo_iso es de: {((df.isnull().sum().iloc[0]/df.shape[0])*100):.2f}%")
print(f"La cantidad de proporcion de datos nulos en anio es de: {((df.isnull().sum().iloc[1]/df.shape[0])*100):.2f}%")
print(f"La cantidad de proporcion de datos nulos en indice es de: {((df.isnull().sum().iloc[2]/df.shape[0])*100):.2f}%")
print(f"La cantidad de proporcion de datos nulos en ranking es de: {((df.isnull().sum().iloc[3]/df.shape[0])*100):.2f}%")
print(f"La cantidad de proporcion de datos nulos en pais es de: {((df.isnull().sum().iloc[4]/df.shape[0])*100):.2f}%")


La cantidad de proporcion de datos nulos en codigo_iso es de: 0.00%
La cantidad de proporcion de datos nulos en anio es de: 0.00%
La cantidad de proporcion de datos nulos en indice es de: 12.98%
La cantidad de proporcion de datos nulos en ranking es de: 7.33%
La cantidad de proporcion de datos nulos en pais es de: 0.00%


In [284]:
a=((df.isnull().sum().iloc[0]/df.shape[0])*100)
if(a > 30 ):
  print("La proporcion de datos nulos en codigo_iso es mayor al 30%")
b=((df.isnull().sum().iloc[1]/df.shape[0])*100)
if(b > 30 ):
  print("La proporcion de datos nulos en anio es mayor al 30%")
c=((df.isnull().sum().iloc[2]/df.shape[0])*100)
if(c > 30 ):
  print("La de proporcion datos nulos en indice es mayor al 30%")
d=((df.isnull().sum().iloc[3]/df.shape[0])*100)
if(d > 30 ):
  print("La de proporcion datos nulos en ranking es mayor al 30%")
e=((df.isnull().sum().iloc[4]/df.shape[0])*100)
if(e > 30 ):
  print("La proporcion de datos nulos en pais es mayor al 30%")
if(a<30 and b<30 and c<30 and d<30 and e<30):
  print("no hay columnas con mas del 30% de datos faltantes")

no hay columnas con mas del 30% de datos faltantes


In [285]:
numpais = df['pais'].nunique()
print("la cantidad de paises distintos son",numpais)
numanio= df['anio'].nunique()
print("la cantidad de anio distintos son",numanio)

la cantidad de paises distintos son 179
la cantidad de anio distintos son 17


In [286]:
print(f"El DataFrame contiene {df.duplicated().sum()} filas duplicadas.")

El DataFrame contiene 0 filas duplicadas.


In [287]:
print("No deberian haber inconsistencias, pues ya la eliminamos antes en la parte 1)b) donde vimos si es que dos paises tenian el mismo codigo_iso o en cambio si un pais tenia dos codigos_iso")

No deberian haber inconsistencias, pues ya la eliminamos antes en la parte 1)b) donde vimos si es que dos paises tenian el mismo codigo_iso o en cambio si un pais tenia dos codigos_iso





### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [288]:
# respuesta
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
       'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
       'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
       'USA', 'VEN']
#por que esta usa??
df_america =  pd.DataFrame()
latam =pd.Series([False]*len(df))
for n in america:
  latam= latam| (df['codigo_iso']==n)
df_latam= df[latam]
anios=df['anio'].astype(str).str.strip().str.split(' ').str[0].unique()
years=anios.astype(int)
for n in years:
  df_anio=df_latam[df_latam['anio']==n]
  paisminlatam = df_anio[df_anio['indice'] == df_anio['indice'].min()]
  print(f"El/los pais/es que es minimo en indice en latam en el anio {n} es/son:")
  print(paisminlatam)
  paismaxlatam = df_anio[df_anio['indice'] == df_anio['indice'].max()]
  print(f"El/los pais/es que es maximo en indice en latam en el anio {n} es/son:")
  print(paismaxlatam)
#En 2016 no hay info de indice

El/los pais/es que es minimo en indice en latam en el anio 2001 es/son:
   codigo_iso  anio  indice  ranking    pais
27        CAN  2001     0.8      2.0  Canadá
El/los pais/es que es maximo en indice en latam en el anio 2001 es/son:
   codigo_iso  anio  indice  ranking  pais
39        CUB  2001    90.3     99.0  Cuba
El/los pais/es que es minimo en indice en latam en el anio 2002 es/son:
    codigo_iso  anio  indice  ranking               pais
341        TTO  2002     1.0      2.0  Trinidad y Tobago
El/los pais/es que es maximo en indice en latam en el anio 2002 es/son:
    codigo_iso  anio  indice  ranking  pais
218        CUB  2002   97.83    125.0  Cuba
El/los pais/es que es minimo en indice en latam en el anio 2003 es/son:
    codigo_iso  anio  indice  ranking               pais
520        TTO  2003     2.0     30.0  Trinidad y Tobago
El/los pais/es que es maximo en indice en latam en el anio 2003 es/son:
    codigo_iso  anio   indice  ranking       pais
363        ARG  2003  3582

In [289]:
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
       'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
       'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
       'USA', 'VEN']
latam =pd.Series([False]*len(df))
for n in america:
  latam= latam| (df['codigo_iso']==n)
df_latam= df[latam]
agrumin= df_latam.groupby('anio').apply(lambda x: x[x['indice'] == x['indice'].min()])
agrumax= df_latam.groupby('anio').apply(lambda x: x[x['indice'] == x['indice'].max()])
print(agrumin)
print(agrumax)

          codigo_iso  anio  indice  ranking               pais
anio                                                          
2001 27          CAN  2001    0.80      2.0             Canadá
2002 341         TTO  2002    1.00      2.0  Trinidad y Tobago
2003 520         TTO  2003    2.00     30.0  Trinidad y Tobago
2004 699         TTO  2004    2.00     31.0  Trinidad y Tobago
2005 737         BOL  2005    4.50     63.0            Bolivia
     743         CAN  2005    4.50     63.0             Canadá
2006 922         CAN  2006    4.88     84.0             Canadá
2007 1101        CAN  2007    3.33     50.0             Canadá
2008 1280        CAN  2008    3.70     62.0             Canadá
2009 1602        USA  2009    6.75    115.0     Estados Unidos
2012 1692        JAM  2012    9.88    176.0            Jamaica
2013 1871        JAM  2013   10.90      6.0            Jamaica
2014 1996        CAN  2014   10.99      3.0             Canadá
2015 2186        CRI  2015   11.10      2.0         Cos

/tmp/ipython-input-3677329114.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  agrumin= df_latam.groupby('anio').apply(lambda x: x[x['indice'] == x['indice'].min()])
/tmp/ipython-input-3677329114.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  agrumax= df_latam.groupby('anio').apply(lambda x: x[x['indice'] == x['indice'].max()])


### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [290]:
pivot_table= pd.pivot_table(df,values='indice',index='codigo_iso',columns='anio',aggfunc='max',fill_value=0)
pivot_table.head(100)

anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
codigo_iso,,,,,,,,,,,,,,,,
AFG,35.5,40.17,28.25,39.17,44.25,56.50,59.25,54.25,51.67,37.36,37.07,37.44,37.75,39.46,37.28,36.55
AGO,30.2,28.00,26.50,18.00,21.50,26.50,29.50,36.50,28.50,37.80,36.50,37.84,39.89,40.42,38.35,34.96
ALB,0.0,6.50,11.50,14.17,18.00,25.50,16.00,21.75,21.50,30.88,29.92,28.77,29.92,29.92,29.49,29.84
AND,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6.82,6.82,19.87,19.87,21.03,22.21,24.63
ARE,0.0,37.00,50.25,25.75,17.50,20.25,14.50,21.50,23.75,33.49,36.03,36.73,36.73,39.39,40.86,43.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LIE,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,7.35,7.02,17.67,18.36,20.31,20.49,20.49
LKA,15.8,24.83,36.50,33.25,50.75,67.50,78.00,75.00,62.50,56.59,59.13,60.28,44.96,44.34,41.37,39.61
LSO,0.0,17.75,29.50,19.50,16.00,29.50,29.50,27.50,24.00,28.36,28.36,28.36,28.78,28.78,28.78,29.74


In [291]:
pivot_table = pivot_table.replace(0, np.nan)
valormax= pivot_table.max().max()
maximos =pivot_table.max()
buscaanio= maximos[maximos == valormax]
aniomax = buscaanio.index[0]
buscaisomax = pivot_table[aniomax]
isomax = buscaisomax[buscaisomax ==valormax]
maxiso= isomax.index[0]
print(f"El pais que tiene mayor indice es {maxiso} con un indice de {valormax} en el anio {aniomax}")
valormin= pivot_table.min().min()
minimos =pivot_table.min()
buscaaniomin= minimos[minimos == valormin]
aniomin = buscaaniomin.index[0]
buscaisomin = pivot_table[aniomin]
isomin = buscaisomin[buscaisomin ==valormin]
miniso= isomin.index[0]
print(f"El pais que tiene menor indice es {miniso} con un indice de {valormin} en el anio {aniomin}")


El pais que tiene mayor indice es KSV con un indice de 64536.0 en el anio 2014
El pais que tiene menor indice es FIN con un indice de 0.5 en el anio 2001


In [292]:
promaniomax= pivot_table.mean(axis=0).max()
promanio=  pivot_table.mean(axis=0)
aniomaxpro = promanio[ promanio==promaniomax ].index[0]
print(f"El anio con promedio mas alto en indece fue el {aniomaxpro} con un indice de {promaniomax}")
promaniomin= pivot_table.mean(axis=0).min()
minproanio=  pivot_table.mean(axis=0)
aniominpro = minproanio[ minproanio==promaniomin ].index[0]
print(f"El anio con promedio minimo en indece fue el {aniominpro} con un indice de {promaniomin}")

El anio con promedio mas alto en indece fue el 2012 con un indice de 468.88350877192977
El anio con promedio minimo en indece fue el 2002 con un indice de 25.879683544303795


In [293]:
valvarimaypais= pivot_table.max(axis=1) -  pivot_table.min(axis=1)
maycar = (pivot_table.max(axis=1) -  pivot_table.min(axis=1)).max()
paismayvar = valvarimaypais[valvarimaypais == maycar].index[0]
print(f"El pais con mayor variabilidad es {paismayvar} con una variabilidad de {maycar}")

El pais con mayor variabilidad es KSV con una variabilidad de 46098.0


In [294]:
valvariiguapais= pivot_table.max(axis=1) -  pivot_table.min(axis=1)
paisiguavar = valvariiguapais[valvariiguapais == 0]
if(paisiguavar.empty):
  print("No hay valores con indice constante a lo largo del tiempo")
else:
  print("Estos/Este son/es los/el paises/paies con indice constante ")
  print(paisiguavar)

Estos/Este son/es los/el paises/paies con indice constante 
codigo_iso
ATG    0.0
GRD    0.0
dtype: float64


In [295]:
pivot_table_conceros= pd.pivot_table(df,values='indice',index='codigo_iso',columns='anio',aggfunc='max',fill_value=0)
nodatapais= pivot_table_conceros.sum(axis=1)
paissindata = nodatapais[nodatapais == 0]
if(paissindata.empty):
  print("Todos los paises tiene datos")
else:
  print("Estos/Este son/es los/el paises/paies que no tienen dato ")
  print(paissindata)


Todos los paises tiene datos
